<a href="https://colab.research.google.com/github/jl205-maker/Diabetes-Risk-Prediction/blob/main/cis545.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CIS 5450 Final Project - Diabetes/Pre-Diabetes Risk Prediction**

Xuejiao Chen, Yanxin Chen, Jiani Liu

## Project Presentation

https://drive.google.com/file/d/1edsr2jBcjwQM3L0_xKOlMQsdlZVURgaF/view?usp=sharing

This project aims to develop an interpretable model that helps people better understand the key factors associated with diabetes and prediabetes. As healthcare continues to play an increasingly central role in people's lives, we hope our work offers meaningful insights into how behavioral, lifestyle, and socio-demographic variables contribute to chronic disease risk. By applying supervised machine learning techniques and big data analytics, this study highlights the importance of modifiable behaviors and social determinants in early disease detection and prevention.

Our primary dataset is the `CDC Diabetes Health Indicators` dataset from `UCI Machine Learning` Repository, which contains responses from 253,680 U.S. adults who participated in the `CDC BRFSS 2014 survey`. The dataset includes a wide range of features such as `BMI`, `blood pressure`, `cholesterol levels`, `smoking habits`, `physical activity`, `diet`, `mental` and `physical` health, and demographic information including `age`, `sex`, `education`, and `income`. Leveraging these variables, we will build a transparent and practical machine learning model that predicts an individual's risk of diabetes or prediabetes.

In addition to the predictive model, this study provides a reproducible framework that can be extended to broader public health datasets. The goal is to support both patients and healthcare professionals in identifying high-risk individuals and promoting targeted, preventative interventions.

**Project Framework**

Part I: Data Loading & Preprocessing

Part II: Exploratory Data Analysis

Part III: Feature Engineering & Preprocessing

Part IV: Modeling

**Critical Variable Definitions**

The key attributes in this dataset are defined as follows:

* **Target Variable (`Diabetes_binary`):**
    * `0` = No Diabetes
    * `1` = Prediabetes or Diabetes
* **Key Health Indicators:**
    * `HighBP` / `HighChol`: 0 = No, 1 = Yes (Doctor diagnosed).
    * `BMI`: Body Mass Index (Continuous value).
    * `GenHlth`: Self-reported general health on a scale of 1-5 (**1 = Excellent**, 5 = Poor). *Note: Higher values indicate worse health.*
    * `MentHlth` / `PhysHlth`: Days in the past 30 days with poor mental or physical health (Scale 0-30).
* **Lifestyle Factors:**
    * `Smoker`: Have you smoked at least 100 cigarettes in your entire life? (0 = No, 1 = Yes).
    * `PhysActivity`: Any physical activity in past 30 days other than job? (0 = No, 1 = Yes).
    * `Fruits` / `Veggies`: Consume 1 or more times per day (0 = No, 1 = Yes).
* **Demographics:**
    * `Education`: Ordinal scale 1-6 (6 = College Graduate).
    * `Income`: Ordinal scale 1-8 (8 = $75,000 or more).

**Dependencies**

In [ ]:
# ================================
# 1. Install dependencies
# ================================
!pip install ucimlrepo
!pip install pandas numpy seaborn matplotlib scikit-learn
!pip install shap
!pip install imbalanced-learn
import shap
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import plot_importance
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegressionCV
from scipy.stats import randint

# 1. Data Acquisition & preprocessing

##1.1 Load the data

In [ ]:
# ================================
# 2. Load dataset
# ================================
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np

cdc = fetch_ucirepo(id=891)

X = cdc.data.features
y = cdc.data.targets

df = pd.concat([X, y], axis=1)

print("Shape:", df.shape)
df.head(5)


ConnectionError: Error connecting to server

##1.2 Basic verification

In [ ]:
# Remove leading/trailing spaces in column names
df.columns = df.columns.str.strip()

df.info()
df.describe(include='all')
df.isnull().sum()
df.head(5)

Since the dataset shows zero missing values, it' not necessary to apply `.dropna()`.

## 1.3 Context on Attributes (Data Dictionary Code)

In [ ]:
# Create a quick look-up dictionary for the ordinal features to provide context within the notebook
# This helps interpret charts later on (e.g., knowing that GenHlth 5 is 'Poor', not 'Excellent')

attribute_context = {
    "GenHlth": {1: "Excellent", 2: "Very Good", 3: "Good", 4: "Fair", 5: "Poor"},
    "Education": {1: "Never attended", 2: "Elementary", 3: "Some HS", 4: "HS Grad", 5: "Some College", 6: "College Grad"},
    "Income": {1: "<$10k", 2: "<$15k", 3: "<$20k", 4: "<$25k", 5: "<$35k", 6: "<$50k", 7: "<$75k", 8: ">$75k"},
    "Diabetes_binary": {0: "Healthy", 1: "Diabetes/Pre-diabetes"}
}

print("Context check for Ordinal Variables:")
for col, context in attribute_context.items():
    print(f"\n{col} Mapping:")
    print(context)

# 2. Exploratory Data Analysis

In [ ]:
# Descriptive statistics for the dataframe
numeric_features = ['BMI', 'GenHlth', 'Age', 'MentHlth', 'PhysHlth', 'Income', 'Education']
df[numeric_features].describe()

## 2.1 Target distribution

###2.1.1 Distribution of population with/without Diabetes

Understanding the distrubution of the target variable is crucial for supervised task, especially for training classification models. We visualize it by using bar plot.

In [ ]:
sns.countplot(x='Diabetes_binary', data=df)
plt.title("Target Distribution: 0 = No diabetes, 1 = Pre/diabetes")
plt.show()

df['Diabetes_binary'].value_counts(normalize=True)

The visualization shows that the dataset is heavily imbalanced, where positive class (i.e. diabetes = 1) is much smaller than the negative class.

###2.1.2 Distribution by Demographic Groups

In [ ]:
edu_diabetes_rate = df.groupby('Education')['Diabetes_binary'].mean().reset_index()

plt.figure(figsize=(8,5))
sns.barplot(data=edu_diabetes_rate, x='Education', y='Diabetes_binary')
plt.xlabel('Education Level')
plt.ylabel('Diabetes Rate (%)')
plt.title('Proportion of Diabetes by Education Level')
plt.ylim(0, 1)
plt.show()

In [ ]:
inc_diabetes_rate = df.groupby('Income')['Diabetes_binary'].mean().reset_index()

plt.figure(figsize=(8,5))
sns.barplot(data=inc_diabetes_rate, x='Income', y='Diabetes_binary')
plt.xlabel('Income Level')
plt.ylabel('Diabetes Rate (%)')
plt.title('Proportion of Diabetes by Income Level')
plt.ylim(0, 1)
plt.show()

In [ ]:
age_mapping = {
    1: "18–24",
    2: "25–29",
    3: "30–34",
    4: "35–39",
    5: "40–44",
    6: "45–49",
    7: "50–54",
    8: "55–59",
    9: "60–64",
    10: "65–69",
    11: "70–74",
    12: "75–79",
    13: "80+"
}

df['AgeRange'] = df['Age'].map(age_mapping)

age_diabetes_rate = df.groupby('AgeRange')['Diabetes_binary'].mean().reset_index()

plt.figure(figsize=(8,5))
sns.barplot(data=age_diabetes_rate, x='AgeRange', y='Diabetes_binary')
plt.xlabel('Age')
plt.ylabel('Diabetes Rate (%)')
plt.title('Proportion of Diabetes by Age')
plt.ylim(0, 1)
plt.show()
# Drop AgeRange column for model training
df.drop(columns=['AgeRange'], inplace=True)

## 2.2 Correlation

### 2.2.1 Correlation Heatmap

In [ ]:
plt.figure(figsize=(12, 10))
corr = df.corr()
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Correlation Heatmap")
plt.show()

### 2.2.2 Top correlating features with target

In [ ]:
corr_target = corr['Diabetes_binary'].sort_values(ascending=False)
corr_target

Based on the correlation data, we propose the following threshold:



*   0.00-0.10: very weak
*   0.10-0.30: weak
*   0.30-0.50: moderate
*   0.50-0.70: strong
*   0.7-1: very strong





Takeaways: The correlation analysis reveals that diabetes is most strongly associated with factors related to overall metabolic and physical health. Worse general health shows the highest positive correlation with diabetes, followed by high blood pressure, difficulty walking, elevated BMI, and high cholesterol, indicating that cardiometabolic conditions cluster together in individuals with diabetes. Age, history of heart disease or heart attack, poorer physical health, and stroke also show moderate positive associations, reflecting increased vulnerability in older adults and those with cardiovascular complications. In contrast, lifestyle and socioeconomic factors tend to correlate negatively with diabetes. Higher levels of physical activity, greater fruit and vegetable consumption, and higher education and income levels are associated with lower diabetes prevalence, with income showing one of the strongest protective relationships. Overall, these results highlight the combined influence of metabolic health, functional limitations, lifestyle choices, and socioeconomic status in shaping diabetes risk within the population.

## 2.4 Logical Value Checks

In [ ]:
print("BMI range:", df['BMI'].min(), "to", df['BMI'].max())
print("MentHlth range:", df['MentHlth'].min(), "to", df['MentHlth'].max())
print("PhysHlth range:", df['PhysHlth'].min(), "to", df['PhysHlth'].max())

##2.5 Handle Outliers

### Percentile Based Filtering

To address potential outliers in the dataset, we focused specifically on `BMI`, which is the only continuous variable where extreme values may reflect measurement or entry errors rather than meaningful variation. **Based on the boxplot, we can see that BMI distribution is right-skewed** and can include implausibly high or low values, we applied a percentile-based filtering approach to remove only the most unlikely observations. Specifically, we **retained BMI values between the 0.5th and 99.5th percentiles** and excluded values outside this range. This method preserves the realistic distribution of BMI while eliminating biologically improbable extremes that could distort model training. All other variables in the dataset are ordinal or binary, so additional outlier removal was unnecessary.

In [ ]:
plt.figure(figsize = (15,15))
for i,col in enumerate(['BMI']):
    plt.subplot(4,2,i+1)
    sns.boxplot(x = col, data = df)
plt.show()

In [ ]:
df_filtered = df.copy()

# Compute reasonable percentile thresholds for BMI
lower_p = df['BMI'].quantile(0.005)   # 0.5th percentile
upper_p = df['BMI'].quantile(0.995)   # 99.5th percentile

print("Lower bound (0.5th percentile):", lower_p)
print("Upper bound (99.5th percentile):", upper_p)

# Filter out extremely unlikely BMI values
df_filtered = df[(df['BMI'] >= lower_p) & (df['BMI'] <= upper_p)]

print("Original shape:", df.shape)
print("Filtered shape:", df_filtered.shape)

In [ ]:
# Visualize the effect before and after removing outliers using percentile-based cutoff
# Original and filtered BMI
bmi_original = df['BMI']
bmi_filtered = df_filtered['BMI']

plt.figure(figsize=(14,6))

# --- Before filtering ---
plt.subplot(1, 2, 1)
sns.histplot(bmi_original, bins=40, kde=True)
plt.title("BMI Distribution (Before Filtering)")
plt.xlabel("BMI")
plt.ylabel("Count")

# --- After filtering ---
plt.subplot(1, 2, 2)
sns.histplot(bmi_filtered, bins=40, kde=True, color='orange')
plt.title("BMI Distribution (After 0.5th–99.5th Percentile Filtering)")
plt.xlabel("BMI")
plt.ylabel("Count")

plt.tight_layout()
plt.show()

## **2.6 EDA Summary & Project Roadmap**

### **Summary of Findings**
1.  **Target Imbalance:** The dataset is heavily imbalanced, with only ~14% of observations representing diabetic cases. This finding necessitates specific modeling strategies, such as using **Class Weights** (e.g., `scale_pos_weight` in XGBoost or `class_weight='balanced'` in Logistic Regression) and prioritizing metrics like **Recall** and **ROC-AUC** over simple Accuracy.
2.  **Correlation Correlations:**
    * **Positive Correlations:** `GenHlth` (General Health), `HighBP` (High Blood Pressure), and `BMI` have the strongest positive correlations with diabetes. Note that since `GenHlth` scales 1-5 (Excellent to Poor), a positive correlation confirms that poorer self-reported health links to diabetes.
    * **Negative Correlations:** `Income`, `Education`, and `PhysActivity` act as protective factors (negative correlation).
3.  **Data Integrity:** `BMI` contained extreme outliers (up to 98), which were filtered using percentile capping to prevent skewing distance-based models.

### **Updated Project Roadmap**
Based on these insights, the modeling phase will proceed as follows:

1.  **Baseline Model (Logistic Regression):** We will establish a baseline using a linear classifier with class weights to handle the imbalance.
2.  **Dimensionality Reduction (PCA):** Due to multicollinearity among health metrics (e.g., `PhysHlth` vs `GenHlth`), we will test if PCA improves model stability, though we risk losing interpretability.
3.  **Feature Engineering:** We will create domain-specific features (e.g., `CardioRiskScore`) to capture the interaction between comorbidities (BP, Cholesterol) which EDA showed are clustered together.
4.  **Patient Segmentation (Unsupervised Learning):** We will employ K-Means clustering to uncover latent patient archetypes (risk profiles) and use these clusters as new features to capture complex multi-dimensional patterns.
5.  **Non-Linear Models (XGBoost & Random Forest):** Since health risks often have thresholds (non-linear relationships), we will deploy tree-based models which generally handle ordinal data and interactions better than logistic regression.

# 3. Preprocessing & Initial Feature Engineering

There is no need for `one-hot encoding` in this dataset because all categorical variables are already numerically encoded. Since the model can directly interpret these encoded values, no additional transformation is required.

##3.1 Drop columns with weak correlations

Logistic Regression Model is sensitive to multicollinearity, and it works best when features are not redundant and have monotonic relationships. Therefore, before applying StandardScaler and PCA, irrelevant columns, include `NoDocbcCost`, `Sex`, and `AnyHealthcare` are dropped.

In [ ]:
# Drop any features that fall within +/- 0.05 corr range
weak_lifestyle_features = ['NoDocbcCost', 'Sex', 'AnyHealthcare']
# Build a cleaned df for Logistic Regression
df_cleaned = df_filtered.drop(columns=weak_lifestyle_features)
df_cleaned.info()

## 3.3 Train/Validation/Test Split Helper (70/15/15)

Given that our ML model requires training data, validation data for hyperparameter tuning, and test data, we decided to employ 70/15/15 train/validation/test split. To produce reproducible results, we set the seed to be 42 to mimic the pseudo-random behavior that we elicited in the homeworks.

In [ ]:
# reusable data stratification function
def stratified_train_val_test_split(X, y, train_size=0.7, val_size=0.5, random_state=42):
  # 70% training
  X_train, X_temp, y_train, y_temp = train_test_split(
      X, y, test_size=1-train_size, random_state=random_state, stratify=y
  )

  # 15% val, 15% test
  X_val, X_test, y_val, y_test = train_test_split(
      X_temp, y_temp, test_size=1-val_size, random_state=random_state, stratify=y_temp
  )

  print("Train:", X_train.shape)
  print("Val:", X_val.shape)
  print("Test:", X_test.shape)

  return (X_train, X_val, X_test, y_train, y_val, y_test)

## 3.4 Scaling

To prepare the data for model training, we applied `standard scaling` to all features using the StandardScaler. Although tree-based models such as `Random Forest` and `XGBoost` are generally insensitive to feature scaling, `logistic regression` is highly affected by differences in feature magnitude because it relies on gradient-based optimization. Scaling ensures that all features contribute proportionally to the model, prevents variables with larger numeric ranges from dominating the loss function, and improves convergence when using class weights. For consistency across models and to maintain a uniform preprocessing pipeline, the validation and test sets were transformed using the scaler fitted only on the training data to avoid data leakage. This standardized scaling process helps stabilize training and ensures fair comparison across all three classifiers.

In [ ]:
# load dataset
X = df_cleaned.drop(columns=['Diabetes_binary'])
y = df_cleaned['Diabetes_binary']

# get stratified dataset
X_train, X_val, X_test, y_train, y_val, y_test = stratified_train_val_test_split(X, y, train_size=0.7, val_size=0.15, random_state=42)

# initialize a standard scaler
scaler = StandardScaler()

# scale data
X_train_cleaned_scaled = scaler.fit_transform(X_train)
X_val_cleaned_scaled = scaler.transform(X_val)
X_test_cleaned_scaled = scaler.transform(X_test)

# 4. Modeling

## 4.1 Weighted Logistic Regression Model

### 4.1.1 Train on Raw Cleaned Dataset

In [ ]:
# ==============================
# Weighted logistic regression model on raw cleaned dataset
# ==============================

log_reg = LogisticRegression(
    penalty = "l2",
    class_weight = "balanced",
    max_iter = 1000,
    solver="liblinear")

# fit model on PCA-transformed training dataset
log_reg.fit(X_train_cleaned_scaled, y_train)

# evaluation using validation set
wlr_y_pred = log_reg.predict(X_val_cleaned_scaled)
wlr_y_prob = log_reg.predict_proba(X_val_cleaned_scaled)[:,1]

# print the evaluation metrics
print("Weighted Logistic Regression on Raw Cleaned Dataset")
print("Accuracy:", accuracy_score(y_val, wlr_y_pred))
print("Precision:", precision_score(y_val, wlr_y_pred))
print("Recall:", recall_score(y_val, wlr_y_pred))
print("F1 Score:", f1_score(y_val, wlr_y_pred))
print("ROC-AUC:", roc_auc_score(y_val, wlr_y_prob))

**Handling imbalanced data**: In this dataset, the target variable `Diabetes_binary` is imbalanced as can be seen in Section 2.1.1, meaning the number of individuals without diabetes (class 0) is much larger than the number with diabetes (class 1).
To address this imbalance, we applied class weighting during model training that assigns a higher penalty to misclassifying the minority class, encouraging the model to pay more attention to positive diabetes cases. For Logistic Regression, this was implemented using class_weight='balanced', which automatically scales weights inversely proportional to class frequencies.

### 4.1.2 PCA for Dimensionality Reduction

To address potential multicollinearity among predictors and to explore classical dimensionality-reduction techniques, we then apply `Principal Component Analysis (PCA)`, retaining components that capture 95% of the variance. This PCA-based logistic regression model serves as an initial benchmark and illustrates how transforming the data into orthogonal components can improve numerical stability while sacrificing interpretability. Building on this foundation, we later introduce domain-informed feature engineering to construct a richer and more clinically meaningful representation of the data, ultimately comparing the performance of PCA-driven optimization against models that preserve interpretability through thoughtfully designed features.

In [ ]:
# Fit PCA to retain 95% variance
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_cleaned_scaled)
X_val_pca = pca.transform(X_val_cleaned_scaled)
X_test_pca = pca.transform(X_test_cleaned_scaled)

# Print the number of components retained
print(f"Number of components retained: {pca.n_components_}")
print(X_train_pca.shape, X_val_pca.shape)

### 4.1.2 Train on PCA-Transformed Dataset


In [ ]:
# ==============================
# Weighted logistic regression model on PCA-transformed features
# ==============================

log_reg_pca = LogisticRegression(
    penalty = "l2",
    class_weight = "balanced",
    max_iter = 1000,
    solver="liblinear")

# fit model on PCA-transformed training dataset
log_reg_pca.fit(X_train_pca, y_train)

# evaluation using validation set
wlr_pca_y_pred = log_reg_pca.predict(X_val_pca)
wlr_pca_y_prob = log_reg_pca.predict_proba(X_val_pca)[:,1]

# print the evaluation metrics
print("Weighted Logistic Regression on PCA-Transformed Features")
print("Accuracy:", accuracy_score(y_val, wlr_pca_y_pred))
print("Precision:", precision_score(y_val, wlr_pca_y_pred))
print("Recall:", recall_score(y_val, wlr_pca_y_pred))
print("F1 Score:", f1_score(y_val, wlr_pca_y_pred))
print("ROC-AUC:", roc_auc_score(y_val, wlr_pca_y_prob))

**Main Takeaways**

The baseline weighted logistic regression model trained on the raw cleaned dataset achieved an `accuracy` of 0.735, `recall` of 0.747, `F1-score` of 0.439, and a `ROC-AUC` of 0.823. After applying PCA to retain 95% of the variance, the logistic regression model trained on the PCA-transformed features showed small improvements in `accuracy`, `precision`, and `recall`, with an `F1-score` of 0.441. However, PCA resulted in a slight decrease in `ROC-AUC` (0.820), indicating a reduction in the model's ability to discriminate between positive and negative cases. Overall, PCA provided only **marginal performance gains while sacrificing interpretability**. These results motivated the development of a third model using domain-informed engineered features, with the goal of achieving stronger predictive performance while preserving clinical interpretability.

## 4.2 More Feature Engineering - New Composite Features

PCA successfully compresses correlated features into orthogonal components and often enhances model stability. However, while PCA produced a reasonable baseline model, it also revealed a critical drawback: the resulting components no longer corresponded to meaningful health or lifestyle variables. This loss of interpretability limited our ability to explain which factors contribute most to diabetes risk, an essential consideration for real-world applications. Recognizing this trade-off, we shifted our approach from purely mathematical transformations toward a more **domain-informed** strategy.

### 4.2.1 Cardiometabolic Risk Score
`HighBP`, `HighChol`, `BMI`, and `HeartDiseaseorAttack` all relate to metabolic syndrome. Diabetes strongly correlates with cardiovascular/metabolic comorbidities.


In [ ]:
df_cleaned_2 = df_cleaned.copy()

df_cleaned_2['CardioRiskScore'] = (
    df_cleaned_2['HighBP'] +
    df_cleaned_2['HighChol'] +
    df_cleaned_2['HeartDiseaseorAttack']
)

### 4.2.2 Overall Health Burden Score

`GenHlth + MentHlth + PhysHlth` summarize daily limitations. High symptom burden predicts chronic disease.

In [ ]:
df_cleaned_2['HealthBurden'] = df_cleaned_2['GenHlth'] + df_cleaned_2['MentHlth'] + df_cleaned_2['PhysHlth']

### 4.2.3 Diet Score: Combine fruit/vegetable intake

In [ ]:
# Combine Fruits and Veggies indicators into a composite indicator
df_cleaned_2['DietScore'] = df_cleaned_2['Fruits'] + df_cleaned_2['Veggies']

# Drop Fruits and Veggies
df_cleaned_2.drop(columns=['Fruits', 'Veggies'], inplace=True)

### 4.2.4 HealthLifestyleScore

In [ ]:
# Physical Activity − Heavy Alcohol − Smoker
df_cleaned_2['HealthyLifestyleScore'] = (
    df_cleaned_2['PhysActivity'] -
    df_cleaned_2['Smoker'] -
    df_cleaned_2['HvyAlcoholConsump']
)

### 4.2.5 Income-to-Education Socioeconomic Index

In [ ]:
# Income-to-education socioeconomic index
df_cleaned_2['SES_Index'] = df_cleaned_2['Income'] * 0.7 + df_cleaned_2['Education'] * 0.3

### 4.2.6 Symptom Duration Flags

Clinically, ≥14 days per month is considered “persistent.” Flag chronic symptoms when they last over 14 days.

In [ ]:
df_cleaned_2['ChronicMentHlth'] = (df_cleaned_2['MentHlth'] >= 14).astype(int)
df_cleaned_2['ChronicPhysHlth'] = (df_cleaned_2['PhysHlth'] >= 14).astype(int)

### 4.2.7 Mobility-Limited High Risk Indicator

Captures obesity + mobility limitation → major diabetes risk.

In [ ]:
df_cleaned_2['HighRiskMobility'] = (df_cleaned_2['DiffWalk'] & (df_cleaned_2['BMI'] > 30)).astype(int)


### 4.2.8 Physical Activity × BMI
Low activity + high BMI amplifies diabetes risk.

In [ ]:
# Physical Activity × BMI
# Low activity + high BMI amplifies diabetes risk
df_cleaned_2['BMI_PhysAct_Interaction'] = df_cleaned_2['BMI'] * (1 - df_cleaned_2['PhysActivity'])

### Feature Engineering Summary

To enrich the predictive power of the model, several engineered features were created based on **clinical relevance** and **variable relationships**.These include `DietScore` and `HealthyLifestyleScore`, which summarize overall dietary and lifestyle quality; `SES_Index`, a composite socioeconomic metric combining income and education; and chronic health indicators (`ChronicMentHlth` and `ChronicPhysHlth`) that flag individuals experiencing persistent symptoms. Additional risk-enhancing interactions were introduced, such as the `BMI_PhysAct_Interaction` capturing the combined effect of high BMI and low physical activity, and `HighRiskMobility` reflecting the intersection of mobility difficulty and obesity. Age was further transformed into an ordered categorical grouping (`AgeGroup`) to better represent age-related risk strata. Collectively, these engineered features enhance the dataset by incorporating clinically meaningful relationships, improving interpretability, and providing the models with richer representations of underlying health determinants of diabetes.

In [ ]:
df_cleaned_2.info()
df_cleaned_2.head()

## 4.3 Weighted Logistic Regresion Model Cont.

By engineering clinically relevant composite features and interaction terms, our goal is to construct a new Weighted Logistic Regression model that retained the interpretability necessary for understanding and communicating the underlying drivers of diabetes.

### 4.3.1 Scale the dataset

In [ ]:
# load dataset
X_2 = df_cleaned_2.drop(columns=['Diabetes_binary'])
y_2 = df_cleaned_2['Diabetes_binary']

# get stratified dataset
X_train_2, X_val_2, X_test_2, y_train_2, y_val_2, y_test_2 = stratified_train_val_test_split(X_2, y_2, train_size=0.7, val_size=0.15, random_state=42)

# initialize a standard scaler
scaler = StandardScaler()

# scale data
X_train_cleaned_scaled_2 = scaler.fit_transform(X_train_2)
X_val_cleaned_scaled_2 = scaler.transform(X_val_2)
X_test_cleaned_scaled_2 = scaler.transform(X_test_2)

### 4.3.2 Model Implementation

In [ ]:
# ==============================
# weighted logistic regression model on new engineered dataset
# ==============================
log_reg = LogisticRegression(
    penalty = "l2",
    class_weight = "balanced",
    max_iter = 1000,
    solver="liblinear")

log_reg.fit(X_train_cleaned_scaled_2, y_train_2)

# evaluation using validation set
wlr_y_pred_2 = log_reg.predict(X_val_cleaned_scaled_2)
wlr_y_prob_2 = log_reg.predict_proba(X_val_cleaned_scaled_2)[:,1]

# print the evaluation metrics
print("Weighted Logistic Regression on New Engineered Dataset")
print("Accuracy:", accuracy_score(y_val_2, wlr_y_pred_2))
print("Precision:", precision_score(y_val_2, wlr_y_pred_2))
print("Recall:", recall_score(y_val_2, wlr_y_pred_2))
print("F1 Score:", f1_score(y_val_2, wlr_y_pred_2))
print("ROC-AUC:", roc_auc_score(y_val_2, wlr_y_prob_2))



### 4.3.3 LogisticRegressionCV - Hyperparameter Tuning

We will use the built-in hyperparameter tuning LogisticRegressionCV for tuning the model.

In [ ]:
logreg_cv = LogisticRegressionCV(
    Cs=[0.001, 0.01, 0.1, 1, 10, 100],
    cv=5,
    scoring="roc_auc",
    max_iter=1000,
    class_weight="balanced",
    penalty="l2",
    solver="liblinear"
)

logreg_cv.fit(X_train_cleaned_scaled_2, y_train_2)

In [ ]:
# evaluation using validation set
wlr_y_pred_cv = logreg_cv.predict(X_val_cleaned_scaled_2)
wlr_y_prob_cv = logreg_cv.predict_proba(X_val_cleaned_scaled_2)[:,1]

# print the evaluation metrics
print("Weighted Logistic Regression on New Engineered Dataset, Cross Validated")
print("Accuracy:", accuracy_score(y_val_2, wlr_y_pred_cv))
print("Precision:", precision_score(y_val_2, wlr_y_pred_cv))
print("Recall:", recall_score(y_val_2, wlr_y_pred_cv))
print("F1 Score:", f1_score(y_val_2, wlr_y_pred_cv))
print("ROC-AUC:", roc_auc_score(y_val_2, wlr_y_prob_cv))

By comparing the metrics here with those in section 4.3.2, CV fine-tuning only makes marginal adjustments.

### 4.3.4 Threshold Tuning

In the field of healthcare, false negatives are usually more clinically dangerous than false positives as false negatives will delay treatment, monitoring, and intervention. Therefore, in addition to hyperparameter optimization, we will perform decision threshold tuning on the predicted probabilities. The default classification threshold of 0.5 is often suboptimal, especially in impanced healthcare settings where the costs of false negatives (missed diabetes cases) and false poitives are asymmetric. Rather than changing the model itself, threshold tuning will adjust the cutoff that better balances precision and recall. We will use the validation set to choose a threshold that reduce the probability of false negatives.

In [ ]:
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    precision_recall_curve
)

# predicted probabilities (positive class = 1) on validation set
y_val_prob = logreg_cv.predict_proba(X_val_cleaned_scaled_2)[:, 1]

precisions, recalls, pr_thresholds = precision_recall_curve(y_val_2, y_val_prob)

target_recall = 0.75
best_thresh = 0.5
best_f1 = -1

for p, r, t in zip(precisions, recalls, np.append(pr_thresholds, 1.0)):
    if r >= target_recall:
        y_val_pred_t = (y_val_prob >= t).astype(int)
        f1 = f1_score(y_val_2, y_val_pred_t)
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = t

print("Chosen threshold (recall ≥ 0.75):", best_thresh)
print("F1 at chosen threshold:", best_f1)


In [ ]:
# test probabilities
y_test_prob = logreg_cv.predict_proba(X_test_cleaned_scaled_2)[:, 1]

# apply tuned threshold
y_test_pred = (y_test_prob >= best_thresh).astype(int)

print("=== Final Test Performance: Weighted LR + Tuned Threshold ===")
print("Confusion matrix (test):")
print(confusion_matrix(y_test_2, y_test_pred))
print(classification_report(y_test_2, y_test_pred))
print("ROC-AUC (test, from probabilities):", roc_auc_score(y_test_2, y_test_prob))


Threshold tuning gives us a threshold of 0.49, which shifts the decision boundary to increase sensitivity (recall) for the positive class, thereby reducing false negatives at the cost of increased false positives.

### 4.3.5 WLR Evaluation and Takeaway

Across the three logistic regression models tested, results show a clear trade-off between predictive performance and interpretability. The baseline model trained on the raw cleaned dataset achieved solid performance with a ROC-AUC of 0.823. Applying PCA produced small gains in accuracy, precision, recall, and F1-score, although ROC-AUC decreased slightly to 0.820, indicating that PCA did not meaningfully improve the model’s discriminative ability while substantially reducing interpretability. Finally, the model trained on the engineered feature dataset achieved performance nearly identical to the baseline model (ROC-AUC 0.823), demonstrating that domain-informed composite features and interaction terms can maintain competitive predictive power without sacrificing interpretability. While PCA offered marginal improvements in some metrics, the engineered model provides a more clinically meaningful and transparent representation of diabetes risk factors, making it the more practical choice despite comparable performance.

In [ ]:
# Training set predictions
train_pred = log_reg.predict(X_train_cleaned_scaled_2)
train_prob = log_reg.predict_proba(X_train_cleaned_scaled_2)[:,1]

print("Training Performance")
print("Training Accuracy:", accuracy_score(y_train_2, train_pred))
print("Training ROC-AUC:", roc_auc_score(y_train_2, train_prob))

# Compare with validation metrics


**Model Performance Summary**

By comparing the training and validation performance of the Weighted Logistic Regression model (model 3), we noticed that it shows nearly identical performance on the training and validation sets, with training and validation `ROC-AUC` scores of 0.8232 and 0.8227, respectively. This indicates that the model is not overfitting and generalizes well to unseen data. The moderate overall performance further suggests mild underfitting, which is expected given the linear nature of logistic regression in a complex, nonlinear health dataset. Despite this, the model remains stable, interpretable, and suitable for understanding the contribution of various health and lifestyle factors to diabetes risk.

## 4.4 Patient Segmentation (Unsupervised Learning)

Before moving to non-linear models like XGBoost, we employ **K-Means Clustering**, an unsupervised learning algorithm, to uncover hidden patient archetypes in the dataset.

**Goal:** Group individuals with similar multi-dimensional health profiles (e.g., combining BMI, Age, and General Health) into distinct clusters.
**Relevance:** The resulting cluster assignment (`Patient_Cluster`) serves as a powerful new feature. It encapsulates complex, non-linear relationships between health variables that manual feature engineering might miss. Feeding this cluster information into downstream models (XGBoost) can help them better identify high-risk groups.

In [ ]:
from sklearn.cluster import KMeans

# 1. Initialize and Fit K-Means
# We use the scaled data for clustering to ensure all features contribute equally to distance calculations.
# k=3 is chosen to represent potentially low, medium, and high-risk groups.
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans.fit(X_train_cleaned_scaled_2)

# 2. Generate Cluster Labels
# We predict the cluster for Train, Validation, and Test sets
train_clusters = kmeans.labels_
val_clusters = kmeans.predict(X_val_cleaned_scaled_2)
test_clusters = kmeans.predict(X_test_cleaned_scaled_2)

# 3. Add 'Patient_Cluster' as a New Feature
# We add this directly to our dataframe so XGBoost can use it as a predictor.
X_train_2['Patient_Cluster'] = train_clusters
X_val_2['Patient_Cluster'] = val_clusters
X_test_2['Patient_Cluster'] = test_clusters

print("Feature Engineering Complete: 'Patient_Cluster' column added to datasets.")

# 4. Visualize the Risk Profile of Each Cluster
# This step validates if the clusters are meaningful regarding Diabetes risk.
cluster_risk = X_train_2.copy()
cluster_risk['Diabetes_binary'] = y_train_2

plt.figure(figsize=(8, 5))
sns.barplot(x='Patient_Cluster', y='Diabetes_binary', data=cluster_risk, hue='Patient_Cluster', palette='viridis', legend=False)
plt.title('Diabetes Risk by Unsupervised Patient Cluster')
plt.ylabel('Diabetes Probability')
plt.xlabel('Cluster Label')
plt.show()

**Observation of Clusters:**
The K-Means algorithm successfully stratified the population into three distinct risk profiles without ever seeing the target variable (`Diabetes_binary`). As shown in the bar chart above:
* **Cluster 2 (Low Risk):** Represents a group with a very low diabetes prevalence (~5%). These individuals likely represent the "healthy baseline" (e.g., younger age, lower BMI, fewer comorbidities).
* **Cluster 0 (Moderate Risk):** Shows a moderate diabetes prevalence (~22%).
* **Cluster 1 (High Risk):** Represents the highest risk group, with a prevalence of nearly 30%. This cluster likely captures the confluence of multiple risk factors (e.g., Age + HighBP + HighChol) that we identified in the EDA phase.

**Strategic Value for Modeling:**
The stark difference in diabetes probability across these clusters confirms that `Patient_Cluster` acts as a **strong proxy for overall health status**. By treating this cluster label as a new categorical feature, we provide the downstream supervised models (XGBoost) with a high-level "summary" of a patient's risk profile, potentially simplifying the decision boundary and improving predictive performance.

##4.5 XGBoost Classifier

While the weighted logistic regression models demonstrated stable performance, their predictive capacity is fundamentally limited by their linear nature. The evaluation metrics indicates that there exists underfitting. The decision boundary cannot fully capture the complexity, non-linear relationship among health, lifestyle, and socioeconomic variables in the dataset. This drawback motivates more expressive non-linear models. In this section, we will explore XGBoost to model non-linear interactions, automatically discover feature combinations, and handle class imbalance through built-in parameter tuning.

### 4.5.1 Handling Class Imbalance

Since the distribution of the target variable is heavily imblanced (people without diabetes significantly outnumber those with diabetes), we compute the `class ratio` and use it as the `scale_pos_weight`. From the computation, we observe that for every 1 diabetic individual (class 1), there are 6.2 non-diabetic individuals (class 0). Thus, model will predict "no diabetes" most of the time, and Precision/Recall for positive cases will drop. Therefore, we tell the XGBoost model to increase the loss impact of positive class proportional to the imbalance. This will help XGB avoid being biased toward class 0.

In [ ]:
# Get the class counts for the target variable
class_counts = y_train_2.value_counts()
# Calculate the class ratio to handle class imbalance
# class_ratio = (# negative)/(# positive)
class_ratio = class_counts[0] / class_counts[1]
print(f'Target variable has {class_ratio:.2f}x more observations in the negative (majority) class than the positive (minority) class.')

### 4.5.2 XGBoost Implementation

PCA is unnecessary for XGBoost because decesion trees do not rely on linear feature interactions and are robust to multicollinearity. Therefore, we will use the domain-informed dataset for training the XGBoost model.

In [ ]:
'''
XGBoost Classifier Parameters
- n_estimators = 300: enough complexity without overfitting
- scale_pos_weight = class_ratio: handles class imbalance without resampling
- max_depth = 4: prevents overly complex trees, avoid overfitting
- subsample = 0.8: adds randomness to reduce overfitting
- colsample_bytree = 0.8: uses 80% of features per tree to reduce correlation
- learning_rate = 0.05: small learning rate for stable and smooth learning
'''
# Initialize an XGB Claassifier with default params (train 1)
xgb_clf = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    eval_metric='logloss',
    scale_pos_weight=class_ratio,
    n_jobs=-1,
    random_state=42,
    enable_categorical=True  # Optional: helps if explicitly cast clusters to category type
)

# Fit the classifier to the data
xgb_clf.fit(X_train_2, y_train_2)

# Predict the target variable for the training/validation datasets
xgb_y_train_pred = xgb_clf.predict(X_train_2)
xgb_y_val_pred = xgb_clf.predict(X_val_2)
xgb_y_val_prob = xgb_clf.predict_proba(X_val_2)[:,1]

### 4.5.3 XGBoost Evaluation

In [ ]:
print("XGBoost on Engineered Dataset")
print("Accuracy:", accuracy_score(y_val_2, xgb_y_val_pred))
print("Precision:", precision_score(y_val_2, xgb_y_val_pred))
print("Recall:", recall_score(y_val_2, xgb_y_val_pred))
print("F1 Score:", f1_score(y_val_2, xgb_y_val_pred))
print("ROC-AUC:", roc_auc_score(y_val_2, xgb_y_val_prob))

The XGBoost Classifier clearly outperforms the logistic regression baselines. It achieves the highest ROC-AUC (0.829), indicating the strongest ability so far to discriminate between diabetic and non-diabetic individuals. Notably, it also secures the highest Recall (78.5%), suggesting that it correctly identifies significantly more diabetic cases than both PCA-Logistic Regression and previous LR-Engineered models. This sensitivity is critical for a health-risk prediction model where missing a positive case is costly. Furthermore, it maintains the highest **F1 score** observed so far.

### 4.5.4 Feature Importance Plot (XGBoost)

A visualization demonstrating the top-15 most influential engineered features.

In [ ]:
xgb.plot_importance(xgb_clf, max_num_features=15, height=0.5)
plt.title("XGBoost Feature Importance (Top 15)")
plt.show()


### 4.5.5 SHAP Explaination

While XGBoost provides the strongest prediction performance in this study so far, its boosted-tree structure makes it inherently less interpretable than regression models. Because interpretability is crucial in a health-related context, we will employ `SHAP (SHapley Additive exPlanations)` to understand how individual features influence the model's predictions. SHAP quantifies the overall importance of each feature across the dataset, visualizes how features increase or decrease predicted diabetes risk, and reveals non-linear patterns and interactions that the model has learned.

In [ ]:
# explain the model's predictions using SHAP
explainer = shap.TreeExplainer(xgb_clf)

# Calculate shap values on the Validation DataFrame
shap_values = explainer.shap_values(X_val_2)

# Summary plot
shap.summary_plot(shap_values, X_val_2, feature_names=X_val_2.columns)

In [ ]:
explanation = explainer(X_val_2)
shap_values = explanation.values
explanation.base_values
np.abs(shap_values.sum(axis=1) + explanation.base_values - xgb_y_val_prob).max()

# Print out the SHAP explanation values
N = 5
i = 0
sorted_idx = np.argsort(np.abs(shap_values[i]))[::-1]

print(f"\n=== Top {N} SHAP Features Effects for Sample {i}\n")
current_columns = X_val_2.columns

for idx in sorted_idx[:N]:
  effect = shap_values[i][idx]
  feature = current_columns[idx]
  if effect > 0:
        print(f"{feature}: increases predicted diabetes risk by {effect:.4f}")
  else:
      print(f"{feature}: decreases predicted diabetes risk by {abs(effect):.4f}")

**SHAP Takeaway**

The SHAP feature importance analysis reveals that self-reported general health (`GenHlth`), `BMI`, `Age`, and the composite cardiometabolic risk score (`CardioRiskScore`) are the strongest drivers of the XGBoost model's predictions. Notably, the unsupervised learning feature `Patient_Cluster` has emerged as a top-5 predictor, confirming its ability to capture complex risk profiles. `HighBP` and socioeconomic status (`SES_Index`) also remain relevant predictors. Overall, the results indicate that the model relies most heavily on **metabolic health indicators**, **general health status**, and **patient archetypes**, while individual lifestyle features play a more modest direct role.

### 4.5.6 RandomizedSearchCV - Hyperparameter Tuning

The hyperparameter combination used in the initial XGBoost model was chosen empirically, and we recognized the potential for further optimization. Therefore, we applied RandomizedSearchCV with cross-validation to systematically explore the hyperparameter space and select a combination that maximizes validation performance while controlling overfitting.

In [ ]:
# Define the parameter grid for XGradient Boosting Classifier
# We will use the fixed/pre-computed class_ratio as scale_pos_weight

param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.3]
}

# Use the model defined in Section 4.2.2 as the base model

# Create Randomized Search
random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_dist,
    n_iter=25,
    cv=3,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# Fit on X_train_2 (The DataFrame with the new feature)
random_search.fit(X_train_2, y_train_2)

print("Best hyperparameters:")
print(random_search.best_params_)

# Best model evaluation
xgb_best = random_search.best_estimator_
y_val_pred = xgb_best.predict(X_val_2)
y_val_prob = xgb_best.predict_proba(X_val_2)[:, 1]
print("\nXGBoost (tuned with RandomizedSearchCV) on validation set")
print("Accuracy:",  accuracy_score(y_val_2, y_val_pred))
print("Precision:", precision_score(y_val_2, y_val_pred))
print("Recall:",    recall_score(y_val_2, y_val_pred))
print("F1 Score:",  f1_score(y_val_2, y_val_pred))
print("ROC-AUC:",   roc_auc_score(y_val_2, y_val_prob))

Based on the CV result, we find the best hyperparameter combination as follows:



*   `learning rate = 0.03` Slower, more controlled boosting that prevents overfitting
*   `subsample = 0.6` More randomness for better generalization
* `min_child_weight = 1` Allows splits on small leaf nodes to capture subtle patterns
* `gamma = 0` No aplit penalty, the tree is allowed to be more flexible.

Other parameters stay the same.

Based on the evaluation metrics, the performance is nearly identical. Note that RandomizedSearchCV is less exhaustive than GridSearchCV because it does not evaluate every combination of hyperparameters, but in this case, it seems as if GridSearchCV would not have made a substantial difference in improving the model (given that GridSearchCV is largely oriented towards smaller datasets).

## 4.6 Random Forest Classifier

We will now explore a different ensemble model: a Random Forest Classifier. Because Random Forests build multiple decision trees, they are scale-invariant. We decided to use the engineered, **unscaled** dataset for this model instead of using the PCA-transformed one to retain interpretability and prediction accuracy.

###4.6.1 Model Implementation & Evaluation

In [ ]:
SEED = 12345
# Initialize the RandomForestClassifier with the above parameters
rfc = RandomForestClassifier(
    class_weight = 'balanced',
    n_estimators = 100,
    max_depth = 5,
    random_state = SEED)

# Fit it to the training set
rfc.fit(X_train_2, y_train_2)

# Predict the target variable for the training/validation datasets
rfc_y_train_pred = rfc.predict(X_train_2)
rfc_y_val_pred = rfc.predict(X_val_2)
rfc_y_val_prob = rfc.predict_proba(X_val_2)[:,1]

In [ ]:
print("Random Forest Classifier (Engineered Dataset)")
print("Accuracy:",  accuracy_score(y_val_2, rfc_y_val_pred))
print("Precision:", precision_score(y_val_2, rfc_y_val_pred))
print("Recall:",    recall_score(y_val_2, rfc_y_val_pred))
print("F1 Score:",  f1_score(y_val_2, rfc_y_val_pred))
print("ROC-AUC:",   roc_auc_score(y_val_2, rfc_y_val_prob))

### 4.6.2 RandomizedSearchCV - Hyperparameter Tuning

In [ ]:
from scipy.stats import randint


In [ ]:
# Hyperparameter search space
param_dist_rf = {
    'n_estimators': randint(50, 200),
    'max_depth': [3, 5, 7],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True]
}

# RandomizedSearchCV
rf_search = RandomizedSearchCV(
    estimator=rfc,
    param_distributions=param_dist_rf,
    n_iter=25,
    scoring='roc_auc',
    cv=3,
    n_jobs=1,
    verbose=1,
    random_state=SEED
)

rf_search.fit(X_train_2, y_train_2)

# Best model
rf_best = rf_search.best_estimator_
print("Best Random Forest Hypers:", rf_search.best_params_)

In [ ]:
rf_val_pred = rf_best.predict(X_val_2)
rf_val_prob = rf_best.predict_proba(X_val_2)[:, 1]

print("Random Forest (Tuned) Performance")
print("Accuracy:", accuracy_score(y_val_2, rf_val_pred))
print("Precision:", precision_score(y_val_2, rf_val_pred))
print("Recall:", recall_score(y_val_2, rf_val_pred))
print("F1 Score:", f1_score(y_val_2, rf_val_pred))
print("ROC-AUC:", roc_auc_score(y_val_2, rf_val_prob))


###4.6.3 Feature Importance Plot

In [ ]:
importances = rf_best.feature_importances_
indices = np.argsort(importances)[::-1]
features = X_train_2.columns

top_n = min(10, len(features))

plt.figure(figsize=(10, 6))
plt.title("Random Forest Feature Importance")
plt.bar(range(top_n), importances[indices][:top_n])
plt.xticks(range(top_n), features[indices][:top_n], rotation=45, ha='right')
plt.tight_layout()
plt.show()


# Model Evaluation and Comparison

## Metrics Used and Their Interpretation

In this project, we evaluate all models using five standard metrics: Accuracy, Precision, Recall, F1 Score, and ROC-AUC.

Accuracy reflects the overall correctness of predictions, but because our dataset is highly imbalanced, it is not sufficient on its own.

Precision measures how reliable the model’s positive predictions are, while Recall captures how effectively it detects actual diabetes cases—an especially important consideration in healthcare.

The F1 Score provides a balanced view by combining Precision and Recall into a single metric.

 ROC-AUC evaluates the model’s ability to distinguish between positive and negative classes across all thresholds and serves as the most robust performance indicator in this imbalanced classification setting.

## 5.1 Best baseline model -- *Logistic* Regression

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ----------------------------
# Collect metrics for 3 models
# ----------------------------

metrics_data = {
    "Model": [
        "WLR (Raw Cleaned Data)",
        "WLR + PCA",
        "WLR (Engineered Features)"
    ],
    "Accuracy": [
        accuracy_score(y_val, wlr_y_pred),
        accuracy_score(y_val, wlr_pca_y_pred),
        accuracy_score(y_val_2, wlr_y_pred_cv)
    ],
    "Precision": [
        precision_score(y_val, wlr_y_pred),
        precision_score(y_val, wlr_pca_y_pred),
        precision_score(y_val_2, wlr_y_pred_cv)
    ],
    "Recall": [
        recall_score(y_val, wlr_y_pred),
        recall_score(y_val, wlr_pca_y_pred),
        recall_score(y_val_2, wlr_y_pred_cv)
    ],
    "F1 Score": [
        f1_score(y_val, wlr_y_pred),
        f1_score(y_val, wlr_pca_y_pred),
        f1_score(y_val_2, wlr_y_pred_cv)
    ],
    "ROC-AUC": [
        roc_auc_score(y_val, wlr_y_prob),
        roc_auc_score(y_val, wlr_pca_y_prob),
        roc_auc_score(y_val_2, wlr_y_prob_cv)
    ]
}

df_metrics = pd.DataFrame(metrics_data)


df_melted = df_metrics.melt(id_vars="Model", var_name="Metric", value_name="Score")


plt.figure(figsize=(12,7))
sns.barplot(data=df_melted, x="Metric", y="Score", hue="Model", palette="viridis")

plt.title("Comparison of Logistic Regression Models on Validation Metrics", fontsize=14)
plt.ylabel("Score")
plt.ylim(0, 1)
plt.legend(title="Model")
plt.grid(axis='y', linestyle='--', alpha=0.4)

plt.show()


In [ ]:
import pandas as pd

# Store metrics for each model
metrics_table = pd.DataFrame({
    "Model": [
        "WLR (Raw Cleaned Data)",
        "WLR + PCA",
        "WLR (Engineered Features, CV)"
    ],
    "Accuracy": [
        accuracy_score(y_val, wlr_y_pred),
        accuracy_score(y_val, wlr_pca_y_pred),
        accuracy_score(y_val_2, wlr_y_pred_cv)
    ],
    "Precision": [
        precision_score(y_val, wlr_y_pred),
        precision_score(y_val, wlr_pca_y_pred),
        precision_score(y_val_2, wlr_y_pred_cv)
    ],
    "Recall": [
        recall_score(y_val, wlr_y_pred),
        recall_score(y_val, wlr_pca_y_pred),
        recall_score(y_val_2, wlr_y_pred_cv)
    ],
    "F1 Score": [
        f1_score(y_val, wlr_y_pred),
        f1_score(y_val, wlr_pca_y_pred),
        f1_score(y_val_2, wlr_y_pred_cv)
    ],
    "ROC-AUC": [
        roc_auc_score(y_val, wlr_y_prob),
        roc_auc_score(y_val, wlr_pca_y_prob),
        roc_auc_score(y_val_2, wlr_y_prob_cv)
    ]
})

metrics_table


Overall, the three weighted logistic regression models show very similar performance across all metrics, suggesting that the data is not strongly linearly separable and that the models—given their similar structure—behave almost the same.

The baseline model (Raw Cleaned Data) delivers solid results, with an Accuracy of about 0.735, Recall around 0.747, and a ROC-AUC of 0.823. The strong Recall is especially meaningful here, as it indicates that the model is able to identify a large portion of true diabetes cases—an important requirement in any health-risk prediction setting.

When PCA is added, the model shows slight improvements in Accuracy, Precision, and Recall, and a small increase in F1. This suggests that PCA helps reduce multicollinearity and stabilizes the model to some degree. However, its ROC-AUC drops slightly (0.823 → 0.820), showing that PCA does not actually improve the model’s ability to distinguish between positive and negative cases. Since PCA also removes interpretability, these small gains do not outweigh the loss of meaning behind each feature.

Finally, the model built with engineered features performs almost identically to the baseline model, with ROC-AUC remaining at 0.823 and Recall staying consistently high. Although Accuracy and Precision dip slightly, the differences are minimal. Most importantly, this version of the model is much easier to interpret, as the engineered features—such as cardiometabolic risk scores and health burden indicators—carry clear clinical meaning. Compared with the PCA model, this approach maintains performance while offering far stronger interpretability.


Although the three weighted logistic regression models perform very similarly overall, the model built with engineered features is the most practical and well-balanced version. It matches the baseline model in ROC-AUC, maintains a strong Recall, and avoids the loss of interpretability introduced by PCA. While PCA offers slight numerical gains, the engineered-feature model provides meaningful, clinically interpretable predictors, making it the most suitable Logistic Regression baseline for comparison against more complex nonlinear models such as XGBoost and Random Forest.

## 5.2 Compare models

We first built an XGBoost model as our main nonlinear baseline, since it is highly effective for structured health data and can capture complex interactions that logistic regression cannot. The model was trained on the engineered feature dataset, and we handled class imbalance using scale_pos_weight. After training the initial model, we applied RandomizedSearchCV to tune key hyperparameters such as learning rate, tree depth, and number of estimators. The final tuned model showed strong validation performance, and feature importance analysis helped identify the most influential engineered features.

After XGBoost, we trained a Random Forest classifier as a simpler tree-based model for comparison. We used the engineered feature dataset directly, since Random Forest does not require scaling or PCA and can naturally model nonlinear patterns. To address class imbalance, we applied class_weight='balanced'.
We then tuned the model using RandomizedSearchCV, exploring parameters such as the number of trees, tree depth, and minimum sample requirements for splits and leaves. The tuned model was evaluated on the validation set, and we also extracted feature importance scores to understand which engineered features contributed most to the predictions.

Next, we will compare the best-performing weighted logistic regression model, the parameter-adjusted XGBoost model, and the parameter-adjusted random forest model to comprehensively assess the differences in their predictive capabilities.

In [ ]:
import pandas as pd

# --- Logistic Regression (Engineered + CV) ---
lr_metrics = {
    "Accuracy": accuracy_score(y_val_2, wlr_y_pred_cv),
    "Precision": precision_score(y_val_2, wlr_y_pred_cv),
    "Recall": recall_score(y_val_2, wlr_y_pred_cv),
    "F1 Score": f1_score(y_val_2, wlr_y_pred_cv),
    "ROC-AUC": roc_auc_score(y_val_2, wlr_y_prob_cv)
}

# --- XGBoost (Tuned) ---
xgb_metrics = {
    "Accuracy": accuracy_score(y_val_2, y_val_pred),
    "Precision": precision_score(y_val_2, y_val_pred),
    "Recall": recall_score(y_val_2, y_val_pred),
    "F1 Score": f1_score(y_val_2, y_val_pred),
    "ROC-AUC": roc_auc_score(y_val_2, y_val_prob)
}

# --- Random Forest (Tuned) ---
rf_metrics = {
    "Accuracy": accuracy_score(y_val_2, rf_val_pred),
    "Precision": precision_score(y_val_2, rf_val_pred),
    "Recall": recall_score(y_val_2, rf_val_pred),
    "F1 Score": f1_score(y_val_2, rf_val_pred),
    "ROC-AUC": roc_auc_score(y_val_2, rf_val_prob)
}

# Combine into a DataFrame
metrics_df = pd.DataFrame({
    "Logistic Regression (Engineered + CV)": lr_metrics,
    "XGBoost (Tuned)": xgb_metrics,
    "Random Forest (Tuned)": rf_metrics
})

metrics_df


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(12, 6))

metrics = ["Accuracy", "Precision", "Recall", "F1 Score", "ROC-AUC"]

# Convert df to array for plotting
values = metrics_df.values
x = np.arange(len(metrics))
width = 0.25

plt.bar(x - width, values[:,0], width, label='LR (Engineered + CV)')
plt.bar(x,         values[:,1], width, label='XGBoost (Tuned)')
plt.bar(x + width, values[:,2], width, label='Random Forest (Tuned)')

plt.xticks(x, metrics)
plt.ylabel("Score")
plt.title("Model Performance Comparison")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.5)

plt.show()


Overall, XGBoost is the strongest model among the three. It achieves the best scores in Recall, F1, and ROC-AUC, showing that it most accurately identifies diabetic patients, balances false positives and false negatives, and separates high-risk from low-risk individuals. The engineered logistic regression model (with cross-validation) comes next. Even though it cannot model nonlinear patterns as well as XGBoost, its metrics remain strong, and its AUC and F1 scores are very close to those of XGBoost, making it a stable and highly interpretable option. In comparison, the tuned Random Forest performs slightly worse on all metrics, especially AUC and F1, suggesting that it captures complex feature interactions less effectively. Overall, XGBoost delivers the best predictive performance.

From the perspective of model mechanics, the performance differences among the three models align well with their structural characteristics. XGBoost performs the best because its boosting framework allows each tree to correct the errors of the previous one, enabling the model to gradually learn nonlinear patterns and higher-order feature interactions. In our dataset—which includes health behaviors, metabolic indicators, and lifestyle variables—the relationships among features are far from linear. XGBoost can capture these subtle interactions more effectively, which explains why it achieves the highest Recall, F1, and ROC-AUC scores. In contrast, although Random Forest is also tree-based, each tree is trained independently, without the sequential error-correcting mechanism of boosting. This makes it less capable of modeling complex patterns, resulting in slightly lower overall performance. Logistic Regression, though the simplest model, still performs consistently well, especially after incorporating engineered features that carry clinical meaning. While it cannot learn nonlinear relationships, its linear structure provides strong interpretability, making it particularly useful for identifying key risk factors. Therefore, XGBoost offers the best predictive performance, Random Forest ranks slightly lower, and Logistic Regression remains valuable for its transparency and interpretability.

# Summary


This project focuses on predicting diabetes risk using structured health survey data, with the goal of building models that are both accurate and interpretable. These predictions can support public health planning and help individuals understand their own risk. From our exploratory data analysis, we found that metabolic health factors (such as BMI, high blood pressure, and high cholesterol), overall health status, age, and lifestyle behaviors are the most important predictors of diabetes. The dataset is also highly imbalanced, which guided many of our modeling decisions.

Methodologically, we experimented with both linear and nonlinear models, including weighted logistic regression, Random Forest, and XGBoost. We also added PCA, clinically meaningful engineered features, and K-means clustering to strengthen the feature set and improve model performance. The full pipeline covers preprocessing, feature creation, handling class imbalance, model training, hyperparameter tuning, and final evaluation.

Among all models, XGBoost delivered the strongest performance, achieving the highest Recall, F1, and AUC. This suggests that it captures the complex nonlinear interactions present in health data. Random Forest performed reasonably well but remained slightly behind XGBoost. The engineered logistic regression model, although limited in nonlinear modeling ability, still achieved stable results and offered clear interpretability, which is especially valuable in medical applications.

For interpretability, logistic regression provided direct insight into linear risk factors. XGBoost, combined with SHAP analysis, highlighted the importance of features such as general health (GenHlth), BMI, the cardiometabolic risk score, age, and the unsupervised Patient_Cluster. These results support the value of engineered features and clustering.

Overall, the project highlights several key points:

(1) Structured health data contains many nonlinear relationships, so domain-informed feature engineering is essential for strong performance.

(2) In health contexts, accuracy and interpretability must be balanced.

(3) Unsupervised learning can enhance feature representations and improve downstream models.

(4) Class imbalance must be handled carefully, as it greatly affects the detection of positive cases.

The project also has limitations. Survey data can be subjective, we do not have temporal information, and we have not tested more advanced models such as LightGBM. Future work could include adding more clinical variables, using SHAP-driven feature selection, trying more powerful tree-based models, and exploring fairness across different population subgroups to improve generalizability and real-world impact.
